# imports

In [1]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import pickle
import qplib as qp
from qplib import log, na, nk, num



# log

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)




log('trace: this is a trace message')
log('debug: this is a debug message')
log('info: this is an info message')
log('warning: this is a warning message')
log('error: this is an error message')

log()

# qlang





In [10]:

import numpy as np
import pandas as pd
import copy
import re
import qplib as qp

from IPython.display import display
from ipywidgets import widgets, interactive_output, HBox, VBox, fixed, Layout, interact_manual

from qplib.util import log
from qplib.types import _int, _float, _num, _bool, _datetime, _date, _na, _nk, _yn, _type, qpDict
from qplib.pd_util import _check_df, _diff, _format_df, indexQpExtension, seriesQpExtension, dfQpExtension



# if 'cards' not in globals():
#     cards = pd.read_csv('data/cards.csv')
# cards.q('toughness ´r >2 & <5')
# cards.qi()


# df = qp.get_df()
# df.q(
#     r"""
#     # id ´r ?1  ´n test
#     # name ´m ~ x.upper()
#     # is any ´r is any

#     date of birth / age

#     """,
#     diff=None,
#     inplace=False,
#     verbosity=4,
#     )

# df = qp.get_df()
# df.qi()






## value mask

In [11]:

import numpy as np
import pandas as pd
import copy
import re
import qplib as qp

from IPython.display import display
from ipywidgets import widgets, interactive_output, HBox, VBox, fixed, Layout, interact_manual

from qplib.util import log
from qplib.types import _int, _float, _num, _bool, _datetime, _date, _na, _nk, _yn, qpDict
from qplib.pd_util import _check_df, _diff, _format_df, indexQpExtension, seriesQpExtension, dfQpExtension



# df = qp.get_df()
# df.qi()



# diff

In [ ]:

import pandas as pd
import numpy as np
import copy
import os
import datetime

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.pd_util import _format_df, get_dfs
from qplib.util import log, GREEN, RED, ORANGE, GREEN_LIGHT, RED_LIGHT, ORANGE_LIGHT
from qplib.types import _date, _na, qpDict






df_new, df_old = get_dfs()


df_new, df_old = get_dfs()

print('df_new:')
display(df_new)

print('df_old:')
display(df_old)

print('mode=new:')
display(qp.diff(df_new, df_old, mode='new'))

print('mode=new+:')
display(qp.diff(df_new, df_old, mode='new+'))

print('mode=old:')
display(qp.diff(df_new, df_old, mode='old'))

print('mode=mix:')
display(qp.diff(df_new, df_old, mode='mix'))


In [118]:

import pandas as pd
import numpy as np
import copy
from qplib.pd_util import _format_df
from qplib.util import log, GREEN, ORANGE, RED, GREEN_LIGHT, ORANGE_LIGHT, RED_LIGHT


def diff(
    df_new, df_old,
    mode='mix',  returns='df',  #df, summary, str, all, print, <filename.xlsx>
    index_col=0, max_cols=None, max_rows=None,
    prefix_old='old: ', verbosity=3):
    """
 
    """

    if isinstance(df_new, str) and isinstance(df_old, str) \
        and df_new.endswith('.xlsx') and df_old.endswith('.xlsx'):
        df, summary, string = _diff_excel(
            df_new, df_old, mode, returns,
            index_col, max_cols, max_rows,
            prefix_old, verbosity)
        
    
    else:
        if isinstance(df_new, str):
            if df_new.endswith('.csv'):
                df_new = pd.read_csv(df_new)
            if df_new.endswith('.xlsx'):
                df_new = pd.read_excel(df_new)

        if isinstance(df_old, str):
            if df_old.endswith('.csv'):
                df_new = pd.read_csv(df_old)
            if df_old.endswith('.xlsx'):
                df_new = pd.read_excel(df_old)

        if df_new.equals(df_old):
            df = pd.DataFrame()
            summary = {}
            string = 'both dataframes are identical'
        else:
            df, summary = _diff_df(
                df_new, df_old,
                mode, returns,
                max_cols, max_rows,
                prefix_old, verbosity
                )
            string = _diff_str(df_new, df_old, verbosity)


    if returns == 'df':
        return df
    if returns == 'str':
        return string
    if returns == 'summary':
        return summary
    if returns == 'all':
        return df, summary, string
    if returns == 'print':
        print(string)
        return None
    elif returns.endswith('.xlsx') and isinstance(df, dict): 
        with pd.ExcelWriter(returns) as writer:
            summary.to_excel(writer, sheet_name='summary', index=False)
            if index_col:
                index = True
            else:
                index = False

            for sheet, df in df.items():
                df.data['meta'] = df.data['meta'].str.replace('<br>', '\n')
                df.to_excel(writer, sheet_name=sheet, index=index)
        log(f'info: differences saved to "{returns}"', 'qp._diff.diff()', verbosity)
        return df, summary, string
    elif returns.endswith('.xlsx'):
        df.to_excel(returns, index=index_col)
        log(f'info: differences saved to "{returns}"', 'qp._diff.diff()', verbosity)
        return df, summary, string
    else:
        log(f'error: unknown return value: {returns}', 'qp._diff.diff()', verbosity)
        return None
   

def _diff_df(
    df_new, df_old,
    mode='mix', returns='df',
    max_rows=None, max_cols=None, 
    prefix_old='old: ',
    verbosity=3):
    '''
    shows differences between dataframes

    mode:
    'new': returns new dataframe with highlighted value additions, removals and changes
    'new+': also shows old values in columns next to new values
    'old': returns old dataframe with highlighted value additions, removals and changes
    'mix': returns mixture of new and old dataframe with highlighted value additions, removals and changes
    '''

    if max_rows is None:
        max_rows = 200
    if max_cols is None:
        max_cols = 20
    if returns.endswith('.xlsx'):
        newline = '\n'
    else:
        newline = '<br>'


    if not df_new.index.is_unique:
        log('error: index of new dataframe is not unique', 'qp.diff()', verbosity)
    if not df_old.index.is_unique:
        log('error: index of old dataframe is not unique', 'qp.diff()', verbosity)



    #prepare dataframes
    df_new = _format_df(df_new, fix_headers=False, add_metadata=True, verbosity=verbosity)
    df_old = _format_df(df_old, fix_headers=False, add_metadata=True, verbosity=verbosity)



    cols_added = df_new.columns.difference(df_old.columns)
    cols_removed = df_old.columns.difference(df_new.columns)
    cols_shared = df_new.columns.intersection(df_old.columns)

    rows_added = df_new.index.difference(df_old.index)
    rows_removed = df_old.index.difference(df_new.index)
    rows_shared = df_new.index.intersection(df_old.index)

    summary = {
        'cols added': len(cols_added),
        'cols removed': len(cols_removed),
        'rows added': len(rows_added),
        'rows removed': len(rows_removed),
        'vals added': 0,
        'vals removed': 0,
        'vals changed': 0
        }



    #create dfs showing the highlighted changes dependant on mode argument
    if mode in ['new', 'new+']:
        df_diff = copy.deepcopy(df_new)
        df_diff_style = pd.DataFrame('', index=df_diff.index, columns=df_diff.columns)

        #add metadata columns
        if mode == 'new+':
            cols_new = ['meta']
            cols_add = []
            for col in df_diff.columns:
                if not col.startswith(prefix_old) and col != 'meta':
                    cols_new.append(col)
                    cols_new.append(prefix_old + col)

                    if prefix_old + col not in df_diff.columns:
                        cols_add.append(prefix_old + col)

            df_diff = pd.concat([df_diff, pd.DataFrame('', index=df_diff.index, columns=cols_add)], axis=1)
            df_diff_style = pd.concat([df_diff_style, pd.DataFrame('font-style: italic', index=df_diff.index, columns=cols_add)], axis=1)
        
            df_diff = df_diff[cols_new]
            df_diff_style = df_diff_style[cols_new]


        df_diff_style.loc[:, cols_added] = f'background-color: {GREEN}'
        df_diff_style.loc[rows_added, :] = f'background-color: {GREEN}'

        df_diff.loc[rows_added, 'meta'] += 'added row'



    elif mode == 'old':
        df_diff = copy.deepcopy(df_old)
        df_diff_style = pd.DataFrame('', index=df_diff.index, columns=df_diff.columns)
        
        df_diff_style.loc[:, cols_removed] = f'background-color: {RED}'
        df_diff_style.loc[rows_removed, :] = f'background-color: {RED}'

        df_diff.loc[rows_removed, 'meta'] += 'removed row'

    elif mode == 'mix':
        inds_old = df_old.index.difference(df_new.index)
        cols_old = df_old.columns.difference(df_new.columns)

        df_diff = pd.concat([df_new, df_old.loc[:, cols_old]], axis=1)
        df_diff.loc[inds_old, :] = df_old.loc[inds_old, :]

        df_diff_style = pd.DataFrame('', index=df_diff.index, columns=df_diff.columns)

        df_diff_style.loc[:, cols_added] = f'background-color: {GREEN}'
        df_diff_style.loc[:, cols_removed] = f'background-color: {RED}'
        df_diff_style.loc[rows_added, :] = f'background-color: {GREEN}'
        df_diff_style.loc[rows_removed, :] = f'background-color: {RED}'

        df_diff.loc[rows_added, 'meta'] += 'added row'
        df_diff.loc[rows_removed, 'meta'] += 'removed row'

    else:
        log(f'error: unknown mode: {mode}', 'qp.diff()', verbosity)


    #highlight values in shared columns
    #column 0 contains metadata and is skipped
    cols_shared_no_metadata = [col for col in cols_shared if not col.startswith(prefix_old) and col != 'meta']

    df_new_isna = df_new.loc[rows_shared, cols_shared_no_metadata].isna()
    df_old_isna = df_old.loc[rows_shared, cols_shared_no_metadata].isna()
    df_new_equals_old = df_new.loc[rows_shared, cols_shared_no_metadata] == df_old.loc[rows_shared, cols_shared_no_metadata]

    df_added = df_old_isna & ~df_new_isna
    df_removed = df_new_isna & ~df_old_isna
    df_changed = ~df_new_isna & ~df_old_isna & ~df_new_equals_old

    df_diff_style.loc[rows_shared, cols_shared_no_metadata] += df_added.mask(df_added, f'background-color: {GREEN_LIGHT}').where(df_added, '')
    df_diff_style.loc[rows_shared, cols_shared_no_metadata] += df_removed.mask(df_removed, f'background-color: {RED_LIGHT}').where(df_removed, '')
    df_diff_style.loc[rows_shared, cols_shared_no_metadata] += df_changed.mask(df_changed, f'background-color: {ORANGE_LIGHT}').where(df_changed, '')



    df_added_sum = df_added.sum(axis=1)
    df_removed_sum = df_removed.sum(axis=1)
    df_changed_sum = df_changed.sum(axis=1)

    summary['vals added'] += int(df_added_sum.sum())
    summary['vals removed'] += int(df_removed_sum.sum())
    summary['vals changed'] += int(df_changed_sum.sum())

    df_diff.loc[rows_shared, 'meta'] += df_added_sum.apply(lambda x: f'{newline}vals added: {x}' if x > 0 else '')
    df_diff.loc[rows_shared, 'meta'] += df_removed_sum.apply(lambda x: f'{newline}vals removed: {x}' if x > 0 else '')
    df_diff.loc[rows_shared, 'meta'] += df_changed_sum.apply(lambda x: f'{newline}vals changed: {x}' if x > 0 else '')


    if mode == 'new+':
        cols_shared_metadata = [prefix_old + col for col in cols_shared_no_metadata]
        df_all_modifications = (df_added | df_removed | df_changed)
        df_old_changed = df_old.loc[rows_shared, cols_shared_no_metadata].where(df_all_modifications, '')
        df_diff.loc[rows_shared, cols_shared_metadata] = df_old_changed.values


    if returns.endswith('.xlsx'):
        result = df_diff.style.apply(lambda x: _apply_style(x, df_diff_style), axis=None) 
    else:
        if max_cols is not None and max_cols < len(df_diff.columns):
            log(f'warning: showing {max_cols} out of {len(df_diff.columns)} columns', 'qp.diff()', verbosity)
        if max_rows is not None and max_rows < len(df_diff.index):
            log(f'warning: showing {max_rows} out of {len(df_diff.index)} rows', 'qp.diff()', verbosity)

        df_diff = df_diff.iloc[:max_rows, :max_cols]
        df_diff_style = df_diff_style.iloc[:max_rows, :max_cols]

        #replace "<" and ">" with html entities to prevent them from being interpreted as html tags
        cols_no_metadata = [col for col in df_diff.columns if not col.startswith(prefix_old) and col != 'meta']
        if pd.__version__ >= '2.1.0':
            df_diff.loc[:, cols_no_metadata] = df_diff.loc[:, cols_no_metadata].map(lambda x: _try_replace_gt_lt(x))
        else:
            df_diff.loc[:, cols_no_metadata] = df_diff.loc[:, cols_no_metadata].applymap(lambda x: _try_replace_gt_lt(x))

        result = df_diff.style.apply(lambda x: _apply_style(x, df_diff_style), axis=None)

    return result, summary
    

def _diff_excel(file_new='new.xlsx', file_old='old.xlsx',
    mode='new+', returns='df', index_col=0, 
    max_rows=None, max_cols=None, prefix_old='old: ',
    verbosity=3):
    '''
    shows differences between two excel files.

    specs and requs:
    - only sheets with the same name are compared
    - needs a unique column to use as index, or sequential order of records
    - index must be unique
    - index must correspond to the same "item" in both sheets

    if index_col=None:
    - uses sequential numbers as index instead of any given column
    - uniqueness is guaranteed
    - only works if all sheets have the same "items" in the same rows
    '''

    summary = pd.DataFrame(columns=[
        'sheet',
        f'is in new file',
        f'is in old file',
        f'index (first col) is unique in new file',
        f'index (first col) is unique in old file',
        'cols added',
        'cols removed',
        'rows added',
        'rows removed',
        'vals added',
        'vals removed',
        'vals changed',
        ])
    results = {}
    

    #get names of all sheets in the excel files
    sheets_new = pd.ExcelFile(file_new).sheet_names
    sheets_old = pd.ExcelFile(file_old).sheet_names
    
    #iterate over all sheets
    for sheet in sheets_new:
        if sheet in sheets_old:
            if index_col is None:
                df_new = pd.read_excel(file_new, sheet_name=sheet)
                df_old = pd.read_excel(file_old, sheet_name=sheet)
            else:
                df_new = pd.read_excel(file_new, sheet_name=sheet, index_col=index_col)
                df_old = pd.read_excel(file_old, sheet_name=sheet, index_col=index_col)

            result, changes = _diff_df(
                df_new, df_old,
                mode, returns,
                max_rows, max_cols,
                prefix_old, verbosity
                )
            
            results[sheet] = result
        

            idx = len(summary)
            summary.loc[idx, 'sheet'] = sheet
            summary.loc[idx, f'is in new file'] = True
            summary.loc[idx, f'is in old file'] = True
            summary.loc[idx, f'index (first col) is unique in new file'] = df_new.index.is_unique
            summary.loc[idx, f'index (first col) is unique in old file'] = df_old.index.is_unique
            for key, val in changes.items():
                summary.loc[idx, key] = val
            
        else:
            idx = len(summary)
            summary.loc[idx, 'sheet'] = sheet
            summary.loc[idx, f'is in new file'] = True
            summary.loc[idx, f'is in old file'] = False
            summary.loc[idx, f'index (first col) is unique in new file'] = df_new.index.is_unique
    
    #wip
    string = 'no string version of differences available when comparing excel files'

    return results, summary, string


def _diff_str(df_new, df_old, verbosity=3):
    """"
    compares 2 dfs and returns a summary of differences in string form
    """

    if df_new.equals(df_old):
        return 'both dataframes are identical'
    
    idx_shared = df_new.index.intersection(df_old.index)
    cols_shared = df_new.columns.intersection(df_old.columns)


    #different dtypes, headers and indices

    dtypes_new = {}
    dtypes_old = {}
    for col in cols_shared:
        if df_new[col].dtype != df_old[col].dtype:
            dtypes_new[col] = df_new[col].dtype
            dtypes_old[col] = df_old[col].dtype

    result = 'only in df_new:\n'
    result += f'dtypes: {dtypes_new}\n'
    result += f'indices: {df_new.index.difference(df_old.index).values}\n'
    result += f'headers: {df_new.columns.difference(df_old.columns).values}\n'

    result += 'only in df_old:\n'
    result += f'dtypes: {dtypes_old}\n'
    result += f'indices: {df_old.index.difference(df_new.index).values}\n'
    result += f'headers: {df_old.columns.difference(df_new.columns).values}\n'


    #different values

    if len(cols_shared) > 20 or len(idx_shared) > 200:
        log('warning: too many shared columns or indices to show different values', 'qp._diff.diff()', verbosity)
        result += 'too many shared columns or indices to show different values'

    else:
        df_new_shared = df_new.loc[idx_shared, cols_shared]
        df_old_shared = df_old.loc[idx_shared, cols_shared]
        diffs = df_new_shared != df_old_shared

        result += f'\ndifferent values in df_new:\n{df_new_shared[diffs].astype(str).fillna('')}\n'
        result += f'\ndifferent values in df_old:\n{df_old_shared[diffs].astype(str).fillna('')}\n'

    return result


def _try_replace_gt_lt(x):
    if isinstance(x, str):
        return x.replace('<', '&lt;').replace('>', '&gt;')
    elif isinstance(x, type):
        return str(x).replace('<', '&lt;').replace('>', '&gt;')
    else:
        return x
    
def _apply_style(x, df_style):
    return df_style
 

 

new = 'archive/NET_BM Study_01_export_2024-04-11.xlsx'
old = 'archive/NET_BM Study_01_export_2024-03-15.xlsx'
df1, df2 = qp.get_dfs()


diff(new, old, mode='mix', returns='test.xlsx')


110,"adding column ""meta"" at position 0",df.format(),info,2024-09-18 16:28:24.886802


111,"adding column ""meta"" at position 0",df.format(),info,2024-09-18 16:28:24.892944


112,"adding column ""meta"" at position 0",df.format(),info,2024-09-18 16:28:25.260101


113,"adding column ""meta"" at position 0",df.format(),info,2024-09-18 16:28:25.268122


114,"adding column ""meta"" at position 0",df.format(),info,2024-09-18 16:28:25.913567


...

121,"adding column ""meta"" at position 0",df.format(),info,2024-09-18 16:28:26.737598


122,"adding column ""meta"" at position 0",df.format(),info,2024-09-18 16:28:27.049237


123,"adding column ""meta"" at position 0",df.format(),info,2024-09-18 16:28:27.055507


124,"differences saved to ""test.xlsx""",qp._diff.diff(),info,2024-09-18 16:28:36.269441


(<pandas.io.formats.style.Styler at 0x177cf218b30>,
                            sheet is in new file is in old file  \
 0     cycB16_VD_Visits_VD_Visit1           True           True   
 1            VD_Visits_VD_Visit1           True           True   
 2            VD_Visits_VD_Visit2           True           True   
 3    CP_Completion_CP_Completion           True           True   
 4  AE_AdvEvents_AE_AdverseEvents           True           True   
 5           CM_ConMeds_CM_ConMed           True           True   
 6          MH_MedHist_MH_MedHist           True           True   
 
   index (first col) is unique in new file  \
 0                                    True   
 1                                    True   
 2                                    True   
 3                                    True   
 4                                    True   
 5                                    True   
 6                                    True   
 
   index (first col) is unique in old fi

In [91]:
import random

cards_old = pd.read_csv('data/cards.csv')
cards = cards_old.copy()
for col in cards.columns:
    for ind in random.sample(range(len(cards)), 10):
        cards.loc[ind, col] = np.nan


C:\Users\MartinVölkl-GouyaIns\AppData\Local\Temp\ipykernel_43668\2652149747.py:3: DtypeWarning: Columns (2,3,7,12,16,20,23,47,52,53,61,62,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  cards_old = pd.read_csv('data/cards.csv')


In [98]:
a, b = _diff_str(cards, cards_old)

print(a+b)

49,too many shared columns or indices to show different values,qp._diff.diff(),Warning,2024-09-18 15:14:17.097429


only in df_new:
dtypes: {'hasFoil': dtype('float64'), 'hasNonFoil': dtype('float64')}
indices: []
headers: []
only in df_old:
dtypes: {'hasFoil': dtype('int64'), 'hasNonFoil': dtype('int64')}
indices: []
headers: []
too many shared columns or indices to show different values


# pd_util

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import datetime
import qplib as qp

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.util import log, qpDict
from qplib.types import _date, _na









# types

In [67]:
import pandas as pd
import numpy as np
import re



# "bashlike" wrappers

# tests

In [ ]:

#run tests in folder "tests" using pytest and create a test report
!pytest tests --html=tests/test_report.html


In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import pytest
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)


def get_df_simple():
    df = pd.DataFrame({
        'a': [-1, 0, 1],
        'b': [1, 2, 3]
        })
    return df

def get_df_simple_tagged():
    df = pd.DataFrame({
        'meta': ['', '', ''],
        'a': [-1, 0, 1],
        'b': [1, 2, 3]
        })
    df.index = [0, 1, 2]
    return df


def get_df():
    df = pd.DataFrame({
        'ID': [10001, 10002, 10003, 20001, 20002, 20003, 30001, 30002, 30003, 30004, 30005],
        'name': ['John Doe', 'Jane Smith', 'Alice Johnson', 'Bob Brown', 'eva white', 'Frank miller', 'Grace TAYLOR', 'Harry Clark', 'IVY GREEN', 'JAck Williams', 'john Doe'],
        'date of birth': ['1995-01-02', '1990/09/14', '1985.08.23', '19800406', '05-11-2007', '06-30-1983', '28-05-1975', '1960Mar08', '1955-Jan-09', '1950 Sep 10', '1945 October 11'],
        'age': [-25, '30', np.nan, None, '40.0', 'forty-five', 'nan', 'unk', '', 'unknown', 35],
        'gender': ['M', 'F', 'Female', 'Male', 'Other', 'm', 'ff', 'NaN', None, 'Mal', 'female'],
        'height': [170, '175.5cm', None, '280', 'NaN', '185', '1', '6ft 1in', -10, '', 200],
        'weight': [70.2, '68', '72.5lb', 'na', '', '75kg', None, '80.3', '130lbs', '82', -65],
        'bp systole': ['20', 130, 'NaN', '140', '135mmhg', '125', 'NAN', '122', '', 130, '45'],
        'bp diastole': [80, '85', 'nan', '90mmHg', np.nan, '75', 'NaN', None, '95', '0', 'NaN'],
        'cholesterol': ['Normal', 'Highe', 'NaN', 'GOOD', 'n.a.', 'High', 'Normal', 'n/a', 'high', '', 'Normal'],
        'diabetes': ['No', 'yes', 'N/A', 'No', 'Y', 'Yes', 'NO', None, 'NaN', 'n', 'Yes'],
        'dose': ['10kg', 'NaN', '15 mg once a day', '20mg', '20 Mg', '25g', 'NaN', None, '30 MG', '35', '40ml']
        })
    return df


def get_df_tagged():
    df1 = get_df()
    df2 = pd.DataFrame('', index=df1.index, columns=['meta', *df1.columns])
    df2.iloc[:, 1:] = df1.loc[:, :]
    return df2


df = qp.get_df()
display(df)

vals = [
    ('date of birth ´v to datetime ´r =05-11-2007', df.loc[[4], ['date of birth']]),
    ]

for val in vals:
    code, expected = val
    temp = df.q('´s verbosity=3  ´c' + code)
    result = df.loc[temp.index, temp.columns]
    assert result.equals(expected), f'{code}:\n{qp.diff(result, expected, returns='str+')}'
    



# temp

In [1]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)


df = qp.get_df()

df.qi()

In [3]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)

df1, df2 = qp.get_dfs()


TypeError: 'module' object is not callable